In [1]:
pip install duckdb --upgrade

Note: you may need to restart the kernel to use updated packages.


In [1]:
import wrds
import duckdb
import pandas as pd

In [2]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_row',
 'boardex_trial',
 'boardex_uk',
 'boardsmp',
 'bvd',
 'bvd_ama_large',
 'bvd_ama_medium',
 'bvd_ama_small',
 'bvd_ama_verylarge',
 'bvd_amadeus_trial',
 'bvd_bvdbankf',
 'bvd_bvdbankf_trial',
 'bvd_orbis_large',
 'bvd_orbis_medium',
 'bvd_orbis_small',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciq',
 'ciq_common',
 'ciq_keydev',
 'ciq_ratings',
 'ciq_transcripts',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_filings',
 'comp

In [4]:
db.list_tables(library="crsp_q_ccm")

['ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'cst_hist',
 'sechead',
 'sechist']

In [5]:
db.describe_table(library="crsp_q_ccm", table="ccmxpf_lnkhist")

Approximately 118951 rows in crsp_q_ccm.ccmxpf_lnkhist.


,name,nullable,type,comment
0,gvkey,True,VARCHAR(6),Standard and Poor\'\'s Identifier
1,linkprim,True,VARCHAR(1),Primary Link Marker
2,liid,True,VARCHAR(3),Security-level Identifier
3,linktype,True,VARCHAR(2),Link Type Codd
4,lpermno,True,DOUBLE PRECISION,Historical CRSP PERMNO Link to COMPUSTAT Record
5,lpermco,True,DOUBLE PRECISION,Historical CRSP PERMCO Link to COMPUSTAT Record
6,linkdt,True,DATE,First Effective Date of Link
7,linkenddt,True,DATE,Last Effective Date of Link


In [6]:
# First, let's analyze missing values in a sample
sample_query = """
    SELECT *
    FROM crsp_q_ccm.ccmxpf_lnkhist
    LIMIT 1000
"""
sample_df = db.raw_sql(sample_query)

# Analyze missing values
missing_analysis = pd.DataFrame({
    'null_count': sample_df.isnull().sum(),
    'null_percentage': (sample_df.isnull().sum() / len(sample_df) * 100).round(2),
    'unique_values': [sample_df[col].dropna().unique()[:5].tolist() for col in sample_df.columns],
    'unique_null_markers': [sample_df[sample_df[col].isnull()][col].unique().tolist() for col in sample_df.columns]
})

print("Missing value analysis from sample:")
print(missing_analysis)

Missing value analysis from sample:
           null_count  null_percentage  \
gvkey               0              0.0   
linkprim            0              0.0   
liid                0              0.0   
linktype            0              0.0   
lpermno           608             60.8   
lpermco           608             60.8   
linkdt              0              0.0   
linkenddt          51              5.1   

                                               unique_values  \
gvkey               [001000, 001001, 001002, 001003, 001004]   
linkprim                                        [C, P, N, J]   
liid                                 [00X, 01, 99X, 02, 01C]   
linktype                                [NU, LU, NR, LC, LX]   
lpermno        [25881.0, 10015.0, 10023.0, 10031.0, 54594.0]   
lpermco          [23369.0, 6398.0, 22159.0, 6672.0, 20000.0]   
linkdt     [1961-01-01, 1970-09-30, 1970-11-13, 1978-01-0...   
linkenddt  [1970-09-29, 1970-11-12, 1978-06-30, 1983-09-1...   

        

In [7]:
db.describe_table(library="crsp_q_ccm", table="ccmxpf_lnkhist")['type']

Approximately 118951 rows in crsp_q_ccm.ccmxpf_lnkhist.


0          VARCHAR(6)
1          VARCHAR(1)
2          VARCHAR(3)
3          VARCHAR(2)
4    DOUBLE PRECISION
5    DOUBLE PRECISION
6                DATE
7                DATE
Name: type, dtype: object

In [8]:
unique_types = db.describe_table(library="crsp_q_ccm", table="ccmxpf_lnkhist")['type'].astype(str).drop_duplicates()

print(unique_types)

Approximately 118951 rows in crsp_q_ccm.ccmxpf_lnkhist.
0          VARCHAR(6)
1          VARCHAR(1)
2          VARCHAR(3)
3          VARCHAR(2)
4    DOUBLE PRECISION
6                DATE
Name: type, dtype: object


In [15]:
# Create DuckDB connection
duck_conn = duckdb.connect('wrds_data.db')


# Get schema information
schema_info = db.describe_table(library="crsp_q_ccm", table="ccmxpf_lnkhist")

# Get total row count
total_rows = db.raw_sql("SELECT COUNT(*) FROM crsp_q_ccm.ccmxpf_lnkhist").iloc[0,0]
print(f"Total rows to process: {total_rows}")

# Drop existing table if it exists
duck_conn.execute("DROP TABLE IF EXISTS ccmxpf_lnkhist")

# Create table with exact schema mapping
create_table_sql = "CREATE TABLE ccmxpf_lnkhist ("
columns = []
for _, row in schema_info.iterrows():
    col_name = row['name']
    wrds_type = str(row['type']).upper()
    
    if wrds_type == 'DATE':
        duck_type = 'DATE'
    elif wrds_type.startswith('VARCHAR'):
        duck_type = wrds_type
    elif wrds_type == 'SMALLINT':
        duck_type = 'SMALLINT'
    elif wrds_type == 'INTEGER':
        duck_type = 'INTEGER'
    elif 'DOUBLE PRECISION' in wrds_type:
        duck_type = 'DOUBLE'
    else:
        duck_type = 'VARCHAR'
        
    columns.append(f"{col_name} {duck_type}")

create_table_sql += ", ".join(columns) + ")"
duck_conn.execute(create_table_sql)

# Process in chunks
chunksize = 100000
offset = 0

while True:
    query = f"""
        select * 
        from crsp_q_ccm.ccmxpf_lnkhist 
        LIMIT {chunksize} 
        OFFSET {offset}
    """
    
    try:
        chunk = db.raw_sql(query)
        
        if len(chunk) == 0:
            break
        
        # Insert the chunk into DuckDB
        duck_conn.register('temp_chunk', chunk)
        duck_conn.sql('INSERT INTO ccmxpf_lnkhist SELECT * FROM temp_chunk')
        
        offset += chunksize
        print(f"Processed {min(offset, total_rows)} rows out of {total_rows} ({(min(offset, total_rows)/total_rows*100):.2f}%)")
        
    except Exception as e:
        print(f"Error at offset {offset}: {str(e)}")
        raise




Approximately 118951 rows in crsp_q_ccm.ccmxpf_lnkhist.
Total rows to process: 118951
Processed 100000 rows out of 118951 (84.07%)
Processed 118951 rows out of 118951 (100.00%)


AttributeError: module 'duckdb' has no attribute 'disconnect'

In [16]:
duck_conn.close()
db.close()